In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = './log/pggan/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 7000
start_epoch = 0
train_batch = 12
test_batch = 200
lr = 0.04
schedule = [1500, 3000, 4500, 6000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b1/to_style1/cutmix' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = 'fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 2500]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.99
cm_prob_low = 0.1

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(target_dir, '100_shot_pggan_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes, 
                                       override_params={'dropout_rate':0.5, 'drop_connect_rate':0.5})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        r = np.random.rand(1)
        if r < cm_prob:
            source_inputs, source_targets = source_set[batch_idx]
            source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()
            
            prob_delta = cm_prob_init - cm_prob_low
            prob_step = epoch / (epochs+1) * prob_delta
            lam = cm_prob_init - prob_step
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            # teacher model output, prediction
            
            teacher_outputs = nn.Softmax()(teacher_model(inputs))
            pred = teacher_outputs.argmax(dim=-1)
            lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            target_a = pred
            target_b = pred[rand_index]
            
            outputs = student_model(inputs)
            loss_main = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(student_model)
            loss_sp = reg_l2sp(student_model)
            
        else:
            source_inputs, source_targets = source_set[batch_idx]
            source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()
            
            prob_delta = cm_prob_init - cm_prob_low
            prob_step = epoch / (epochs+1) * prob_delta
            lam = cm_prob_init - prob_step
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[rand_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            # teacher model output, prediction
            
            lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
            
            target_a = targets
            target_b = targets[rand_index]
            
            outputs = student_model(inputs)
            loss_main = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(student_model)
            loss_sp = reg_l2sp(student_model)
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
        print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(source_train_loader, train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 20 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    
        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 7000] LR: 0.040000
1/17 Data:2.939 | Batch:5.591 | Total:0:00:03 | ETA:0:01:01 | Loss:0.3138970136642456 | top1:91.66667175292969
2/17 Data:0.001 | Batch:1.093 | Total:0:00:04 | ETA:0:00:37 | Loss:0.20603027939796448 | top1:95.83333587646484
3/17 Data:0.001 | Batch:0.543 | Total:0:00:05 | ETA:0:00:26 | Loss:0.4262411793073018 | top1:91.66666412353516


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


4/17 Data:0.001 | Batch:0.632 | Total:0:00:06 | ETA:0:00:20 | Loss:1.9508511573076248 | top1:81.25
5/17 Data:0.001 | Batch:0.671 | Total:0:00:06 | ETA:0:00:17 | Loss:2.3698655724525453 | top1:75.00000762939453
6/17 Data:0.001 | Batch:1.077 | Total:0:00:07 | ETA:0:00:15 | Loss:2.3621705869833627 | top1:72.22222137451172
7/17 Data:0.001 | Batch:0.372 | Total:0:00:08 | ETA:0:00:12 | Loss:2.3459227170263017 | top1:72.61904907226562
8/17 Data:0.001 | Batch:0.719 | Total:0:00:08 | ETA:0:00:11 | Loss:2.314790315926075 | top1:73.95833587646484
9/17 Data:0.001 | Batch:0.938 | Total:0:00:09 | ETA:0:00:09 | Loss:2.2761651476224265 | top1:75.9259262084961
10/17 Data:0.001 | Batch:0.922 | Total:0:00:10 | ETA:0:00:08 | Loss:2.249246174097061 | top1:76.66667175292969
11/17 Data:0.001 | Batch:0.757 | Total:0:00:11 | ETA:0:00:07 | Loss:2.218560527671467 | top1:78.03030395507812
12/17 Data:0.001 | Batch:0.839 | Total:0:00:12 | ETA:0:00:04 | Loss:2.199291631579399 | top1:78.47222137451172
13/17 Data:0.00

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


39/39 Data:0.000 | Batch:0.208 | Total:0:00:31 | ETA:0:00:00 | Loss:2.274435838063558 | top1:50.94871520996094
161/161 Data:0.000 | Batch:0.844 | Total:0:00:51 | ETA:0:00:00 | Loss:1.787040755013439 | top1:97.21495819091797

Epoch: [2 | 7000] LR: 0.068000
1/17 Data:2.857 | Batch:3.703 | Total:0:00:01 | ETA:0:00:30 | Loss:2.051705837249756 | top1:75.0
2/17 Data:0.001 | Batch:0.657 | Total:0:00:02 | ETA:0:00:19 | Loss:1.9639362692832947 | top1:83.33333587646484
3/17 Data:0.001 | Batch:0.546 | Total:0:00:03 | ETA:0:00:15 | Loss:1.9077008565266926 | top1:86.11111450195312
4/17 Data:0.002 | Batch:0.760 | Total:0:00:03 | ETA:0:00:13 | Loss:1.8724492192268372 | top1:87.5
5/17 Data:0.003 | Batch:0.380 | Total:0:00:04 | ETA:0:00:10 | Loss:1.8433256149291992 | top1:86.66667175292969
6/17 Data:0.001 | Batch:0.747 | Total:0:00:04 | ETA:0:00:10 | Loss:1.827370524406433 | top1:87.5
7/17 Data:0.001 | Batch:0.531 | Total:0:00:05 | ETA:0:00:08 | Loss:1.7790051528385706 | top1:89.28571319580078
8/17 Dat

...

16/17 Data:0.002 | Batch:0.964 | Total:0:00:09 | ETA:0:00:01 | Loss:215.64095783233643 | top1:74.47917175292969

Epoch: [85 | 7000] LR: 0.319914
1/17 Data:2.250 | Batch:2.832 | Total:0:00:01 | ETA:0:00:23 | Loss:2878.823486328125 | top1:83.33332824707031
2/17 Data:0.003 | Batch:0.414 | Total:0:00:01 | ETA:0:00:14 | Loss:2788.2237548828125 | top1:70.83333587646484
3/17 Data:0.001 | Batch:0.277 | Total:0:00:02 | ETA:0:00:10 | Loss:2701.4054361979165 | top1:75.0
4/17 Data:0.002 | Batch:0.537 | Total:0:00:02 | ETA:0:00:09 | Loss:2618.21435546875 | top1:77.08333587646484
5/17 Data:0.001 | Batch:0.420 | Total:0:00:03 | ETA:0:00:08 | Loss:2538.454931640625 | top1:80.00000762939453
6/17 Data:0.004 | Batch:0.460 | Total:0:00:03 | ETA:0:00:07 | Loss:2461.9711100260415 | top1:83.33333587646484
7/17 Data:0.002 | Batch:0.450 | Total:0:00:03 | ETA:0:00:06 | Loss:2388.7768903459823 | top1:80.95238494873047
8/17 Data:0.003 | Batch:0.421 | Total:0:00:04 | ETA:0:00:05 | Loss:2318.4927368164062 | top1:79

10/17 Data:0.001 | Batch:0.623 | Total:0:00:06 | ETA:0:00:05 | Loss:34.980094528198244 | top1:82.50000762939453
11/17 Data:0.001 | Batch:0.522 | Total:0:00:07 | ETA:0:00:04 | Loss:33.99970678849654 | top1:83.33333587646484
12/17 Data:0.001 | Batch:0.389 | Total:0:00:07 | ETA:0:00:04 | Loss:33.07156721750895 | top1:83.33333587646484
13/17 Data:0.001 | Batch:0.692 | Total:0:00:08 | ETA:0:00:03 | Loss:32.16624098557692 | top1:83.33333587646484
14/17 Data:0.002 | Batch:0.563 | Total:0:00:08 | ETA:0:00:02 | Loss:31.31099169594901 | top1:82.14286041259766
15/17 Data:0.003 | Batch:0.701 | Total:0:00:09 | ETA:0:00:02 | Loss:30.483412170410155 | top1:81.66667175292969
16/17 Data:0.001 | Batch:0.589 | Total:0:00:09 | ETA:0:00:01 | Loss:29.680068731307983 | top1:82.29167175292969

Epoch: [90 | 7000] LR: 0.319902
1/17 Data:2.112 | Batch:2.578 | Total:0:00:01 | ETA:0:00:21 | Loss:16.59735870361328 | top1:83.33332824707031
2/17 Data:0.001 | Batch:0.468 | Total:0:00:01 | ETA:0:00:14 | Loss:16.0188088

4/17 Data:0.002 | Batch:0.502 | Total:0:00:02 | ETA:0:00:07 | Loss:1.127481073141098 | top1:91.66667175292969
5/17 Data:0.001 | Batch:0.630 | Total:0:00:02 | ETA:0:00:07 | Loss:1.1880350589752198 | top1:88.33333587646484
6/17 Data:0.002 | Batch:0.506 | Total:0:00:03 | ETA:0:00:07 | Loss:1.2069720427195232 | top1:86.11111450195312
7/17 Data:0.001 | Batch:1.064 | Total:0:00:04 | ETA:0:00:07 | Loss:1.2043820789882116 | top1:83.33333587646484
8/17 Data:0.004 | Batch:0.687 | Total:0:00:05 | ETA:0:00:06 | Loss:1.174151562154293 | top1:84.375
9/17 Data:0.002 | Batch:0.678 | Total:0:00:05 | ETA:0:00:06 | Loss:1.1331420474582248 | top1:86.11111450195312
10/17 Data:0.001 | Batch:0.526 | Total:0:00:06 | ETA:0:00:05 | Loss:1.138931691646576 | top1:85.83333587646484
11/17 Data:0.001 | Batch:0.357 | Total:0:00:06 | ETA:0:00:04 | Loss:1.1300800280137495 | top1:84.8484878540039
12/17 Data:0.001 | Batch:0.505 | Total:0:00:07 | ETA:0:00:04 | Loss:1.1110665152470272 | top1:86.11111450195312
13/17 Data:0.

14/17 Data:0.001 | Batch:0.459 | Total:0:00:07 | ETA:0:00:02 | Loss:277.94913463933125 | top1:57.14285659790039
15/17 Data:0.001 | Batch:0.366 | Total:0:00:08 | ETA:0:00:02 | Loss:313.1010689576467 | top1:56.11111068725586
16/17 Data:0.001 | Batch:0.454 | Total:0:00:08 | ETA:0:00:01 | Loss:340.69207651913166 | top1:56.770835876464844

Epoch: [99 | 7000] LR: 0.319878
1/17 Data:2.030 | Batch:2.454 | Total:0:00:01 | ETA:0:00:19 | Loss:707.0675659179688 | top1:66.66667175292969
2/17 Data:0.002 | Batch:0.689 | Total:0:00:01 | ETA:0:00:14 | Loss:684.8504638671875 | top1:58.333335876464844
3/17 Data:0.002 | Batch:0.581 | Total:0:00:02 | ETA:0:00:12 | Loss:663.4852498372396 | top1:72.22222137451172
4/17 Data:0.002 | Batch:0.541 | Total:0:00:02 | ETA:0:00:10 | Loss:643.0989837646484 | top1:72.91667175292969
5/17 Data:0.003 | Batch:0.391 | Total:0:00:03 | ETA:0:00:09 | Loss:623.5260131835937 | top1:73.33333587646484
6/17 Data:0.003 | Batch:0.520 | Total:0:00:03 | ETA:0:00:08 | Loss:604.748306274

6/17 Data:0.001 | Batch:0.579 | Total:0:00:04 | ETA:0:00:09 | Loss:15.373082955678305 | top1:79.16666412353516
7/17 Data:0.004 | Batch:0.452 | Total:0:00:04 | ETA:0:00:07 | Loss:15.003107479640416 | top1:76.19047546386719
8/17 Data:0.001 | Batch:0.448 | Total:0:00:05 | ETA:0:00:06 | Loss:14.613425374031067 | top1:77.08333587646484
9/17 Data:0.002 | Batch:0.362 | Total:0:00:05 | ETA:0:00:06 | Loss:14.232529428270128 | top1:76.85185241699219
10/17 Data:0.001 | Batch:0.832 | Total:0:00:06 | ETA:0:00:05 | Loss:13.835287380218507 | top1:79.16667175292969
11/17 Data:0.001 | Batch:0.621 | Total:0:00:07 | ETA:0:00:04 | Loss:13.481489181518555 | top1:78.78787994384766
12/17 Data:0.003 | Batch:0.938 | Total:0:00:08 | ETA:0:00:04 | Loss:13.121641318003336 | top1:78.47222137451172
13/17 Data:0.001 | Batch:0.747 | Total:0:00:08 | ETA:0:00:03 | Loss:12.778413699223446 | top1:79.4871826171875
14/17 Data:0.001 | Batch:0.521 | Total:0:00:09 | ETA:0:00:02 | Loss:12.456346103123256 | top1:79.761909484863